# Introduction

In this notebook we calculate Gasteiger Charges with RDKit and display them with XSMILES.

In [1]:
import xsmiles
import json # currently XSMILES widget only accepts the python dict in json-text format
import rdkit # make sure you have RDKit installed to run this notebook: pip install rdkit-pypi
from rdkit import Chem
from rdkit.Chem import AllChem

In [8]:
print("Python RDKit version:", rdkit.__version__) 
print("XSMILES-JupyterLab version:", xsmiles.__version__) 

Python RDKit version: 2021.09.2
XSMILES-JupyterLab version: 0.2.1


In [3]:
# creates a simple XSMILES-Molecule structure, without attributes, with only a single XAI method
def get_xsmiles_simple_molecules(smiles_strings, attributions_vectors, method_name):
    molecules = []
    for i in range(len(smiles_strings)):
        molecule = {
            'string': smiles_strings[i],
            'methods': [
                { 'name': method_name, 'scores': attributions_vectors[i] }, 
            ],
            'attributes': {"ID": i}        
        }
        molecules.append(molecule)
    return molecules

### Compute Gasteiger Charges

In [4]:
# set of smiles
smiles_strings = ["N#Cc1c[nH]cc1c1cccc(Cl)c1Cl", "Cc1c(Br)c2ccc(F)cc2n1S(=O)(=O)c1ncn(S(=O)(=O)N(C)C)n1"]

# create RDKit molecules
mols = [Chem.MolFromSmiles(smiles_string, sanitize=True) for smiles_string in smiles_strings]

# compute charges
for mol in mols:
    AllChem.ComputeGasteigerCharges(mol)

# get atom attributions
charges_attributions = [
    [mols[i].GetAtomWithIdx(j).GetDoubleProp('_GasteigerCharge') for j in range(mols[i].GetNumAtoms())] for i in
    range(len(mols))]


xsmiles_molecules = get_xsmiles_simple_molecules(smiles_strings, charges_attributions, "Gasteiger")

### Set up visualization

In [5]:
view_config = {
    'hideBarChart': False,
    'hideAttributesTable': True, 
    'drawerType': 'RDKitDrawer_black', # Possible values RDKitDrawer (colored) or RDKitDrawer_black (black).
    
}

# Set color domain to [-max, 0, max]
all_abs_scores = [abs(score) for attributions in charges_attributions for score in attributions]
max_abs_scores = max(all_abs_scores)

gradient_config = {     
        'palette': 'PRGn_5',
        'thresholds': [0.5], #default []
        'highlight': True,
        'colorDomain': [-max_abs_scores, 0, max_abs_scores],
      }



### Visualize

In [6]:

xsmiles.XSmilesWidget(molecules=json.dumps(xsmiles_molecules), gradient_config=json.dumps(gradient_config), view_config=json.dumps(view_config) )

XSmilesWidget(gradient_config='{"palette": "PRGn_5", "thresholds": [0.5], "highlight": true, "colorDomain": [-…

In [7]:
# show attributions on structures    XSMILES 0.1.3+
view_config['showScoresOnStructure'] = True

# Note that you need Jupyter Lab to see the visualization - it may not work in other environments
xsmiles.XSmilesWidget(molecules=json.dumps(xsmiles_molecules), gradient_config=json.dumps(gradient_config), view_config=json.dumps(view_config) )

XSmilesWidget(gradient_config='{"palette": "PRGn_5", "thresholds": [0.5], "highlight": true, "colorDomain": [-…